### Theory of bed profile forms implemented in some numpy functions
A bed profile model is required to estimate the bathymetry of a channel in 2D. Here we identify a set of functions to estimate the depth profile from a set of parameters that can be measured iun-situ or (ideally!) only at the side of the channel from drone footage. The angles at the sides of the channel are, according to Savenije 2003 and referenced authors, a good representation of bed form similarities. We therefore take these assumed relationships as starting point for our assessment.


In [ ]:
!pip install seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

sns.set_theme()

## Replication of figure 1 in Savenije 2003

In [ ]:
# let's start with some coordinates from left (0) to right bank

def depth_from_width(s, phi=0.25*np.pi, h_m=10):
    """
    Simplest implementation of Eq. 12 from Savenije 2003, where the first coordinate (zero meters) is the lowest point. We stop at zero as that
    is where we reach the natural levee crest.
    """
    # zero is the lowest point in the stream starting in the middle of the stream
    angle = np.minimum(np.tan(phi)/h_m * s, 0.5*np.pi)
    depth = -np.cos(angle)*h_m
    return depth


s = np.linspace(0, 20, 1000)

depth = depth_from_width(s)

plt.plot(s, depth)
plt.xlabel("Distance from deepest point [m]")
plt.ylabel("Depth +shore [m]")


## Implementation over an entire symmetrical channel

Now we assume that the left coordinate is at the natural levee's left bank and the right at the natural levee's right bank.
We can now no longer use both `h_m` and `phi`. We have to resolve one of them to be able to solve the problem.

In [ ]:
def depth_profile_from_parameters(s, phi=None, h_m=None):
    """
    Solving the entire profile using a symmetrical assumption and either phi or h_m defined
    """
    if phi is None and h_m is None:
        raise ValueError("Either phi or h_m has to be supplied. You supplied nothing.")
    if phi is not None and h_m is not None:
        raise ValueError("Either phi or h_m has to be supplied. You supplied both. Select either one of the two")
    # compute the total width of the channel
    B = s[-1] - s[0]

    if phi is None:
        # h_m was supplied, so resolve for phi
        phi = np.arctan(np.pi*h_m/B)
    else:
        # phi was supplied, so resolve for h_m
        h_m = B*np.tan(phi)/np.pi
    # compute ordinate coordinates (i.e. from center line towards banks)
    _s = s - 0.5*B
    return depth_from_width(_s, phi=phi, h_m=h_m)
    



## Test over a set of natural angles of repose
Let's see what happens with different angles of repose. We should be assuming that milder angles lead to shallower water

In [ ]:

for n, phi in enumerate(np.linspace(0.05*np.pi, 0.25*np.pi, 10)):
    depth = depth_profile_from_parameters(s, phi=phi)
    plt.plot(s, depth, label="{:02d}: phi = {:1.2f}".format(n + 1, phi))
    plt.xlabel("Distance (left to right bank) [m]")
    plt.ylabel("Depth +shore [m]")
plt.axis("equal")
plt.legend(fontsize=8)

### First simple model to include asymmetry.

We can assume that the deepest point is not in the middle by weighing together two solutions, assuming a phi on left and a different phi on right bank.

In [ ]:
# left bank
def depth_profile_left_right_parameters(s, phi_left, phi_right):
    """
    Estimate a depth profile using left and right bank phi parameters with an assumed 
    weighting from left to right bank
    """
    depths_left = depth_profile_from_parameters(s, phi=phi_left)
    # right bank
    depths_right = depth_profile_from_parameters(s, phi=phi_right)
    weights = np.linspace(0, 1, len(s))
    return (1 - weights) * depths_left + weights * depths_right

s = np.linspace(0, 100, 100)
depths = depth_profile_left_right_parameters(s, phi_left=0.3*np.pi, phi_right=0.1*np.pi)
plt.plot(s, depths)
plt.axis("equal")
plt.xlabel("Distance (left to right bank) [m]")
plt.ylabel("Depth +shore [m]")
plt.title("asymmetrical profile shape")

### asymmetry with an independent solution for left and right

In relatively shallow transects, it is not so logical that $\phi_{left}$ still has impact on the shape of the areas close to the right bank and vice versa. We can also make a transectional model that simply assumes the left and right are independent. In this case we can constrain a lot of parameters by modelling them as follows:

For the right-side of the channel, the following applies:

$h_m=\frac{2B_r}{\pi}\tan{\phi_r}$

For left-side, the following applies:

$h_m=\frac{2B_l}{\pi}\tan{\phi_l}$

Because there is only one unique $h_m$, combining these two equations gives a very beautiful dimensionless parameter $\gamma$ which dictates symmetry in case $\gamma=1$. When the value is below 1, the deepest point is more to the left of the middle, whilst if gamma is above one it is more to the right:

$\frac{\tan{\phi_r}}{\tan{\phi_l}}=\frac{B_l}{B_r}=\gamma$

Meaning that if we know the angles of repose, we also know the relative distances from the deepest point, or vice versa. The only missing part is the depth in the deepest point and a measure for the total width. Here we can (or should) assume that the conveyance remains stable over the stretch. So it is much better to use the wetted surface as a parameter. This can then stay the same over the entire reach. We can simply model the absolute angles of repose by assuming a stable levee-to-levee cross section $A$ over the entire domain as follows:

From Savenije et al. 2003 we know (eq. 15) that:

$\bar{h}=\frac{2h_m}{\pi}$

As we model the left and right part of the cross section separately, the cross-sectional surface is:

$A=\bar{h}\left(B_l+B_r\right)$

substitution of the eq. for $\gamma$ gives

$A=hB_r\left(1+\gamma\right)$

and reordering yields the unknown for a given cross section $B_r$. $B_l$ follows from $\gamma$ naturally.

$B_r = \frac{A}{h\left(1+\gamma\right)}$

Let's make the model with the different angles and an estimated deepest point, assuming we know or can optimize the dimensionless parameter $\gamma$, the cross-sectional surface $A$ and a measure for the deepest point $h_m$ from our data.

The critical issue is that we need to have a model to decide where the deepest point is, as this can change over short distances. This is solved later. 


In [ ]:
def depth_profile_left_right_parameters(s, gamma, h_m, A):
    """
    we assume here that the most extreme coordinates are at more or less the same elevation and 
    within the area spanning between both natural levees (they do not have to be at the natural levee per se)
    """
    B = s[-1] - s[0]
    frac_right = 1/(1+gamma)  # TODO include the c-parameter to correct fir B_tilde and B difference.
    
    # make coordinates with zero at the deepest point (negative (positive) left (right) of deepest point
    s_ordinal = s - (1-frac_right) * B
    # estimate the average depth (eq. 15 in Savenije 2003)
    h = 2*h_m / np.pi
    # solve B_r and B_l
    B_r = A/(h*(1+gamma))
    B_l = gamma*B_r
    # compute tan phi left and right
    tan_phi_left = h_m*np.pi/(2*B_l)
    tan_phi_right = h_m*np.pi/(2*B_r)
    tan_phi_right2 = gamma * tan_phi_left
    phi_left = np.arctan(tan_phi_left)
    phi_right = np.arctan(tan_phi_right)
    
    depth = np.zeros(s.shape)

    depth[s_ordinal < 0] = depth_from_width(-s_ordinal[s_ordinal < 0], phi=phi_left, h_m=h_m)
    depth[s_ordinal >= 0] = depth_from_width(s_ordinal[s_ordinal >= 0], phi=phi_right, h_m=h_m)
    # depth = depth_from_width(s_ordinal, phi=phi_left, h_m=h_m)

    return np.minimum(depth, 0)

def plot_asym_depth(gamma, h_m, A):
    # assume a 250m wide grid transect space
    f = plt.figure(figsize=(13, 6))
    s = np.linspace(0, 250, 10000)
    depth = depth_profile_left_right_parameters(s, gamma, h_m, A)
    idx = np.where(depth < -1e-9)[0]
    phi_left2 = np.arctan(-(depth[idx[2]]-depth[idx[1]])/(s[idx[2]]-s[idx[1]]))
    phi_right2 = np.arctan((depth[idx[-2]]-depth[idx[-1]])/(s[idx[-2]]-s[idx[-1]]))
    plt.annotate(
        '$\phi_{left}$' + ': {:1.2f} deg.'.format(phi_left2/np.pi*180),
        xy=(s[idx[2]], depth[idx[2]]),
        xytext=(s[idx[2]], depth[idx[2]]+2),
        arrowprops=dict(facecolor='black', shrink=0.05)
    )
    plt.annotate(
        '$\phi_{right}$' + ': {:1.2f} deg.'.format(phi_right2/np.pi*180),
        xy=(s[idx[-2]], depth[idx[-2]]),
        xytext=(s[idx[-2]], depth[idx[-2]]+2),
        arrowprops=dict(facecolor='black', shrink=0.05)
    )
    idx_deepest = np.where(depth == depth.min())[0][0]
    xerr1 = (s[idx_deepest] - s[idx[0]])/2
    xerr2 = (s[idx[-1]] - s[idx_deepest])/2
    plt.errorbar(
        xerr1 + s[idx[0]], 5,
        xerr=xerr1,
        capsize=5.,
        color="k",
    )
    plt.errorbar(
        xerr2 + s[idx_deepest], 5,
        xerr=xerr2,
        capsize=5.,
        color="k",
    )
    plt.annotate(
        '$B_l$' + ' = {:1.2f}'.format(xerr1*2),
        xy=(s[idx[0]]+ xerr1, 7),
        xytext=(s[idx[0]]+ xerr1, 7),
        horizontalalignment="center"
        # arrowprops=dict(facecolor='black', shrink=0.05)
    )    
    plt.annotate(
        '$B_r$' + ' = {:1.2f}'.format(xerr2*2),
        xy=(s[idx_deepest]+ xerr2, 7),
        xytext=(s[idx_deepest]+ xerr2, 7),
        horizontalalignment="center",
        # arrowprops=dict(facecolor='black', shrink=0.05)
    )
    plt.fill_between(s, depth, color="c", alpha=0.7, label="wetted surface")
    plt.plot([s[idx_deepest], s[idx_deepest]], [-h_m, 0], linestyle='--', color="k", label="deepest point")
    plt.plot(s, depth)
    plt.legend()
    # plt.axis("equal")
    plt.ylabel("Depth [m]")
    plt.xlabel("Ordinal coordinate [m]")
    plt.ylim([-30, 10])

interact(
    plot_asym_depth,
    gamma=(0.2, 1./0.2),  # we limit the range from the minimum y-coordinate to the maximum
    h_m=(5, 30),
    # h_m=31.415,
    A=(100, 2000),
)


### Asymmetrical channel behaviour
A channel bed is usually not entirely symmetrical, so we can also assume that some further constraints may be necessary to assess the width/depth relationship. If we supply the left and right angle, then this does not necessarily mean that the depth can be resolved from that, as they may to a certain degree counter. Hence we have to optimize the location over the cross section where the depth is at maximum.



### Modelling the location of the deepest point (distance from the middle of the grid) over the entire stretch
In a natural stream the location of the deepest point typically meanders with a higher frequency than the meanders of the natural levees. This means that the deepest point moves from the left to the right bank as the river travels downstream. This needs to be simulated in our bathymetric model.

Let's assume that we don't want to randomly pick the location of the deepest point, but instead we want to include it in the entire 2D spatial model of the depths. In this case we would need a longitudinal model for the location of the deepest point. We model it here as a distance from the centerline of the gridded domain, and assume here that the distance from the centerline can be modelled as a simple sine function as follows:

$y_{off}\left(x\right)=\alpha\sin{\left(2\pi\frac{x}{L}-\kappa\right)}$

where $y_{off}$ [m] is the offset from the datum (i.e. the center line) we are looking for, simulated as function of the longitudinal distance $x$ [m], $\alpha$ [m] is the amplitude of the offset, $L$ [m] is a length scale for the frequency angle of the function, and $\kappa$ [rad] the phase offset of the sinusoidal function. $\alpha$, $\kappa$ and $L$ are parameters that ideally are optimized with the available surveyed data points, along with the $\phi_{left}$ and $\phi_{right}$ parameters. We may choose to also constrain some of these parameters at some point, e.g. by allowing a user to say that they know where the dry/wet interface is located. That would fix a few of the parameters.

We implement the function below:



In [ ]:
def deepest_point_y_dist(x, alpha, L, kappa, y_off_mean=0.):
    return y_off_mean + alpha*np.sin(2*np.pi*(x/L)-kappa)
                               
    

Let's try out the function interactively with a certain chosen domain

In [ ]:
def plot_y_off(alpha, L, kappa):  #y_off_mean, A_y, omega, kappa):
    x = np.linspace(0, 100, 10000)  # a 100 meter longitudinal stretch
    plt.plot(
        x, 
        deepest_point_y_dist(
            x,
            alpha,
            L,
            kappa,
        )
    )
    plt.xlim([0, 100])
    plt.ylim([-50, 50])
            
interact(
    plot_y_off,
    alpha=(0, 40),
    L=(0, 200),
    kappa=(0, 2*np.pi)
)

### mapping the longitudinal function over a grid with distances from the middle
Let's assume we have a perfectly rectangular grid, where in each grid cell we have computed the distance of the grid cell with respect to one of the banks. the longitudinal diurection in the grid is from left to right, the transects are from top to bottom. We want to calculate for each grid cell, how far it is from the deepest point

In [ ]:
from matplotlib.colors import ListedColormap

def distance_to_deepest(xi, yi, alpha, L, kappa):
    """
    Computes per grid cell, how far the deepest point is from zero coordinate in the y-direction
    """
    # compute the location of the middle
    y_mean = yi.mean(axis=0)
    
    # first we compute for each coordinate where the deepest point in the given transect is with respect to the y=0 coordinate
    y_disti = deepest_point_y_dist(xi, alpha, L, kappa, y_off_mean=y_mean)
    # then we compute how far in the transect we are per grid cell
    perp_distance = y_disti - yi
    return perp_distance

# Let's try it out with an interactive plot again

def plot_y_off_2d(alpha, L, kappa):
    xax = np.linspace(0, 100, 1000)
    yax = np.linspace(0, 50, 500)
    xi, yi = np.meshgrid(xax, yax)  # xi are our longitudinal distances, yi our latitudinal distances
    perp_dist = distance_to_deepest(
        xi,
        yi,
        alpha,
        L,
        kappa
    )
    palette = sns.color_palette("Spectral", 32) #, as_cmap=True)
    cmap = ListedColormap(colors=palette)
    plt.figure(figsize=(13, 6))
    plt.pcolormesh(
        xi,
        yi,
        perp_dist,
        cmap=cmap,
        vmin=-50,
        vmax=50,
    )
    plt.title("Perpendicular distance from deepest point to center")
    cb = plt.colorbar()
    cb.set_label("distance of deepest point from zero ordinal coordinate [m]")

    plt.xlabel("Longitudinal coordinate [m]")
    plt.ylabel("Ordinal coordinate [m]")
            
interact(
    plot_y_off_2d,
    alpha=(0, 40),
    L=(100, 500),
    kappa=(0, 2*np.pi)
)

### Combining the ordinal model with the longitudinal model
We now have a means to estimate the location of the deepest point within a defined grid with the same elevation on left and right side, now we need to establish a 2D model that uses both concepts to estimate bathymetry. 

Only one thing is missing: we need a proxy parameter for $h_m$, ideally dimensionless, which scales all $h_m$ values, as a function of the variation in width over the entire longitudinal stretch. This implies that as the width of the grid changes, the $h_m$ should follow pace. It should be noted that the width of the grid does not necessarily follow the actual natural-levee-to-natural-levee width (it can but it does not have to). Instead it follows the trajectory of an equal-in-elevation channel form that can reasonably be surveyed. We therefore assume here that the grid width as function of longitudinal distance $\tilde{B}(x)$ is proportional to the levee-to-levee width $B$. I.e.

$\frac{B(x)}{\tilde{B}(x)}=c$

This then also means that we can derive $h_m$ for a given cross section with given $\tilde{B}$ as follows:

$h_m=\frac{A\pi}{2c\tilde{B}}$

$c$ [-] therefore becomes our model parameter which must be optimized on data.

The asymmetry $\gamma$ is defined with our sinusoidal function, thus everything together gives us a 2D model with only 5 parameters to optimize being:

* $A$ [m$^2$]: the wetted surface (assumed constant over the entire longitudinal reach
* $c$ [-]: dimensionless constant defining how $h_m$ relates to the grid width at a given location
* $\alpha$ [m]: the amplitude of the offset from the center of the deepest point
* $L$ [m]: the length scale of the offset of deepest point sinus
* $\kappa$ [rad]: the phase of the deepest point sinus

Let us put this together into a function that simulates depth as a function of these 5 parameters and the location in the grid.


In [ ]:
from matplotlib.colors import LightSource
from matplotlib import cm

def depth_profile_left_right_parameters(s, gamma, h_m, A, c):
    """
    we assume here that the most extreme coordinates are at more or less the same elevation and 
    within the area spanning between both natural levees (they do not have to be at the natural levee per se)
    """
    B = s[-1] - s[0]
    frac_right = 1/(1+gamma)  # TODO include the c-parameter to correct fir B_tilde and B difference.
    
    # make coordinates with zero at the deepest point negative (positive) left (right) of deepest point
    s_ordinal = s - (1-frac_right) * B
    # estimate the average depth (eq. 15 in Savenije 2003)
    h = 2*h_m / np.pi
    # solve B_r and B_l
    B_r = A/(h*(1+gamma))
    B_l = gamma*B_r
    # compute tan phi left and right
    tan_phi_left = h_m*np.pi/(2*B_l)
    tan_phi_right = h_m*np.pi/(2*B_r)
    tan_phi_right2 = gamma * tan_phi_left
    phi_left = np.arctan(tan_phi_left)
    phi_right = np.arctan(tan_phi_right)
    
    depth = np.zeros(s.shape)
    # import pdb;pdb.set_trace()
    
    depth[s_ordinal < 0] = depth_from_width(-s_ordinal[s_ordinal < 0], phi=phi_left[s_ordinal < 0], h_m=h_m[s_ordinal < 0])
    depth[s_ordinal >= 0] = depth_from_width(s_ordinal[s_ordinal >= 0], phi=phi_right[s_ordinal >= 0], h_m=h_m[s_ordinal >= 0])
    # depth = depth_from_width(s_ordinal, phi=phi_left, h_m=h_m)
    # return s_ordinal
    return np.minimum(depth, 0)

def depth_2d(xi, yi, alpha, L, kappa, c, A):
    """
    
    """
    # # get the asymmetry parameter
    # gamma = get_gamma(xi, yi, c, alpha, L, kappa)
    # get the h_m values
    B_grid = yi[-1] - yi[0]
    h_m = A*np.pi/(2*c*B_grid)*np.ones(xi.shape)
    gamma = get_gamma(xi, yi, c, alpha, L, kappa, A, h_m)
    depth = depth_profile_left_right_parameters(yi, gamma, h_m, A, c)
    return depth


def get_gamma(xi, yi, c, alpha, L, kappa, A, h_m):
    # compute the location of the middle
    y_mean = yi.mean(axis=0)
    
    # first we compute for each coordinate where the deepest point in the given transect is with respect to the y=0 coordinate
    y_disti = deepest_point_y_dist(xi, alpha, L, kappa, y_off_mean=0.)
    # y_disti = deepest_point_y_dist(xi, alpha, L, kappa, y_off_mean=y_mean) - y_mean
    h = 2*h_m / np.pi

    # compute the grid width (B tilde)
    B_grid = yi[-1] - yi[0]
    Br_grid = B_grid - (y_mean - yi[0] + y_disti.max(axis=0))

    gamma = A/(h*Br_grid*c) - 1
    # gamma = y_disti/(c*B_grid) + 1
    # then we compute how far in the transect we are per grid cell
    return gamma


xax = np.linspace(0, 500, 5000)
yax = np.linspace(0, 150, 500)
xi, yi = np.meshgrid(xax, yax)  # xi are our longitudinal distances, yi our latitudinal distances

A = 100
kappa=0
L=450
alpha=30
c = 0.6
dist = distance_to_deepest(xi, yi, alpha, L, kappa)
f, axs = plt.subplots(ncols=3, figsize=(13, 7))

palette = sns.color_palette("Spectral", 32) #, as_cmap=True)
cmap = ListedColormap(colors=palette)


depth = depth_2d(xi, yi, alpha, L, kappa, c, A)
B_grid = yi[-1] - yi[0]
h_m = A*np.pi/(2*c*B_grid)*np.ones(xi.shape)

gamma = get_gamma(xi, yi, c, alpha, L, kappa, A, h_m)

ls = LightSource(235, 45)

rgb = ls.shade(depth, cmap=cm.gist_earth, vert_exag=5, blend_mode="soft")
p1 = axs[0].pcolormesh(xi, yi, gamma)
axs[0].set_title("Asymmetry $\gamma$ [-]")
plt.colorbar(p1, ax=axs[0])

p2 = axs[1].pcolormesh(xi, yi, dist, cmap=cmap)
axs[1].set_title("Distance center to deepest [m]")
plt.colorbar(p2, ax=axs[1])

p3 = axs[2].pcolormesh(xi, yi, rgb) #, cmap="Blues")
# p3 = axs[2].pcolormesh(xi, yi, depth, cmap=cmap)
axs[2].set_title("Depth below natural levee [m]")
# plt.colorbar(p3, ax=axs[2])



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm
from matplotlib.ticker import LinearLocator


def set_axes_equal(ax):
    """
    Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    """

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(20, 10))

# Plot the surface.
ax.set_box_aspect([1.0, 1.0, 1.0])
surf = ax.plot_surface(
    xi,
    yi,
    depth,
    # rstride=10,
    # cstride=10,
    facecolors=rgb,
    linewidth=0,
    antialiased=False
)

# ax.view_init(30, 15)
scale_x = 1
scale_y = 1
scale_z = 0.25
ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([scale_x, scale_y, scale_z, 1]))